# Chapter 4: Crystal Lattices and Unit Cells

## Learning Objectives
- Understand the concept of a Bravais lattice
- Master lattice vectors and lattice parameters
- Convert between fractional and Cartesian coordinates
- Understand the reciprocal lattice and its applications
- Work with the 14 Bravais lattices and 7 crystal systems

---

## 4.1 What is a Crystal?

A **crystal** is a solid where atoms are arranged in a highly ordered, repeating pattern extending in all three spatial dimensions.

Key concepts:
- **Lattice**: An infinite array of points with translational symmetry
- **Basis**: The group of atoms associated with each lattice point
- **Crystal structure** = Lattice + Basis

### Example: NaCl (Rock Salt)
- **Lattice**: Face-centered cubic (FCC)
- **Basis**: One Na at (0,0,0) and one Cl at (0.5,0.5,0.5)

## 4.2 Lattice Vectors

A 3D lattice is defined by three **lattice vectors** $\mathbf{a}$, $\mathbf{b}$, $\mathbf{c}$.

Any lattice point can be reached by:
$$\mathbf{R} = n_1 \mathbf{a} + n_2 \mathbf{b} + n_3 \mathbf{c}$$

where $n_1, n_2, n_3$ are integers.

### Lattice Parameters

Instead of specifying the full vectors, we often use **lattice parameters**:
- $a, b, c$: Lengths of the lattice vectors
- $\alpha$: Angle between $\mathbf{b}$ and $\mathbf{c}$
- $\beta$: Angle between $\mathbf{a}$ and $\mathbf{c}$
- $\gamma$: Angle between $\mathbf{a}$ and $\mathbf{b}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import List, Tuple, Optional
from dataclasses import dataclass

print("NumPy version:", np.__version__)

In [ ]:
@dataclass
class LatticeParameters:
    """Crystallographic lattice parameters."""
    a: float  # Length of a vector (Å)
    b: float  # Length of b vector (Å)
    c: float  # Length of c vector (Å)
    alpha: float  # Angle between b and c (degrees)
    beta: float   # Angle between a and c (degrees)
    gamma: float  # Angle between a and b (degrees)
    
    def __repr__(self):
        return (f"LatticeParameters(a={self.a:.3f}, b={self.b:.3f}, c={self.c:.3f}, "
                f"α={self.alpha:.1f}°, β={self.beta:.1f}°, γ={self.gamma:.1f}°)")

def lattice_vectors_from_parameters(params: LatticeParameters) -> np.ndarray:
    """Convert lattice parameters to lattice vectors.
    
    Convention (following IUCr):
    - a vector along x-axis
    - b vector in the xy-plane
    - c vector determined by angles
    
    Returns:
        3x3 matrix where rows are a, b, c vectors
    """
    # Convert angles to radians
    alpha = np.radians(params.alpha)
    beta = np.radians(params.beta)
    gamma = np.radians(params.gamma)
    
    # a is along x
    a_vec = np.array([params.a, 0, 0])
    
    # b is in the xy-plane
    bx = params.b * np.cos(gamma)
    by = params.b * np.sin(gamma)
    b_vec = np.array([bx, by, 0])
    
    # c is determined by alpha and beta
    cx = params.c * np.cos(beta)
    cy = params.c * (np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma)
    cz = np.sqrt(params.c**2 - cx**2 - cy**2)
    c_vec = np.array([cx, cy, cz])
    
    return np.array([a_vec, b_vec, c_vec])

def lattice_parameters_from_vectors(lattice_vectors: np.ndarray) -> LatticeParameters:
    """Convert lattice vectors to lattice parameters."""
    a_vec, b_vec, c_vec = lattice_vectors
    
    a = np.linalg.norm(a_vec)
    b = np.linalg.norm(b_vec)
    c = np.linalg.norm(c_vec)
    
    alpha = np.degrees(np.arccos(np.dot(b_vec, c_vec) / (b * c)))
    beta = np.degrees(np.arccos(np.dot(a_vec, c_vec) / (a * c)))
    gamma = np.degrees(np.arccos(np.dot(a_vec, b_vec) / (a * b)))
    
    return LatticeParameters(a, b, c, alpha, beta, gamma)

# Example: Simple cubic
cubic_params = LatticeParameters(a=4.0, b=4.0, c=4.0, alpha=90, beta=90, gamma=90)
cubic_vectors = lattice_vectors_from_parameters(cubic_params)

print("Simple Cubic Lattice:")
print(f"Parameters: {cubic_params}")
print(f"Vectors:\n{cubic_vectors}")

# Example: Hexagonal
hex_params = LatticeParameters(a=3.0, b=3.0, c=5.0, alpha=90, beta=90, gamma=120)
hex_vectors = lattice_vectors_from_parameters(hex_params)

print("\nHexagonal Lattice:")
print(f"Parameters: {hex_params}")
print(f"Vectors:\n{np.round(hex_vectors, 4)}")

## 4.3 The Seven Crystal Systems

Based on symmetry constraints, there are exactly **7 crystal systems**:

| System | Constraints | Example |
|--------|-------------|--------|
| Cubic | $a=b=c$, $\alpha=\beta=\gamma=90°$ | NaCl, Diamond |
| Tetragonal | $a=b\neq c$, $\alpha=\beta=\gamma=90°$ | TiO₂ (rutile) |
| Orthorhombic | $a\neq b\neq c$, $\alpha=\beta=\gamma=90°$ | Sulfur |
| Hexagonal | $a=b\neq c$, $\alpha=\beta=90°$, $\gamma=120°$ | Graphite, ZnO |
| Trigonal | $a=b=c$, $\alpha=\beta=\gamma\neq 90°$ | Calcite, Quartz |
| Monoclinic | $a\neq b\neq c$, $\alpha=\gamma=90°\neq\beta$ | Gypsum |
| Triclinic | No constraints | K₂Cr₂O₇ |

In [ ]:
def classify_crystal_system(params: LatticeParameters, tol: float = 0.1) -> str:
    """Determine the crystal system from lattice parameters."""
    a, b, c = params.a, params.b, params.c
    alpha, beta, gamma = params.alpha, params.beta, params.gamma
    
    # Check angle constraints
    all_90 = all(np.isclose(angle, 90, atol=tol) for angle in [alpha, beta, gamma])
    ab_90_g120 = (np.isclose(alpha, 90, atol=tol) and 
                  np.isclose(beta, 90, atol=tol) and 
                  np.isclose(gamma, 120, atol=tol))
    ag_90 = (np.isclose(alpha, 90, atol=tol) and np.isclose(gamma, 90, atol=tol))
    all_equal_not_90 = (np.isclose(alpha, beta, atol=tol) and 
                        np.isclose(beta, gamma, atol=tol) and 
                        not np.isclose(alpha, 90, atol=tol))
    
    # Check length constraints
    a_eq_b = np.isclose(a, b, rtol=0.01)
    b_eq_c = np.isclose(b, c, rtol=0.01)
    a_eq_c = np.isclose(a, c, rtol=0.01)
    all_equal = a_eq_b and b_eq_c
    
    if all_equal and all_90:
        return "Cubic"
    elif a_eq_b and not b_eq_c and all_90:
        return "Tetragonal"
    elif not a_eq_b and not b_eq_c and not a_eq_c and all_90:
        return "Orthorhombic"
    elif a_eq_b and ab_90_g120:
        return "Hexagonal"
    elif all_equal and all_equal_not_90:
        return "Trigonal (Rhombohedral)"
    elif ag_90 and not np.isclose(beta, 90, atol=tol):
        return "Monoclinic"
    else:
        return "Triclinic"

# Test classification
test_systems = [
    LatticeParameters(4.0, 4.0, 4.0, 90, 90, 90),      # Cubic
    LatticeParameters(4.0, 4.0, 6.0, 90, 90, 90),      # Tetragonal
    LatticeParameters(3.0, 4.0, 5.0, 90, 90, 90),      # Orthorhombic
    LatticeParameters(3.0, 3.0, 5.0, 90, 90, 120),     # Hexagonal
    LatticeParameters(4.0, 4.0, 4.0, 80, 80, 80),      # Trigonal
    LatticeParameters(3.0, 4.0, 5.0, 90, 100, 90),     # Monoclinic
    LatticeParameters(3.0, 4.0, 5.0, 70, 80, 100),     # Triclinic
]

print("Crystal System Classification:")
for params in test_systems:
    system = classify_crystal_system(params)
    print(f"{system:25s} <- {params}")

## 4.4 The 14 Bravais Lattices

Within the 7 crystal systems, there are exactly **14 distinct Bravais lattices** when we consider centering:

- **P (Primitive)**: Points only at cell corners
- **I (Body-centered)**: Additional point at cell center
- **F (Face-centered)**: Additional points at face centers
- **C (Base-centered)**: Additional points at centers of two opposite faces

In [ ]:
def get_centering_vectors(centering: str) -> np.ndarray:
    """Get the fractional coordinates of lattice points for a given centering.
    
    Args:
        centering: 'P' (primitive), 'I' (body-centered), 
                   'F' (face-centered), 'C' (base-centered)
    
    Returns:
        Nx3 array of fractional coordinates
    """
    if centering == 'P':
        return np.array([[0, 0, 0]])
    elif centering == 'I':
        return np.array([
            [0, 0, 0],
            [0.5, 0.5, 0.5]
        ])
    elif centering == 'F':
        return np.array([
            [0, 0, 0],
            [0.5, 0.5, 0],
            [0.5, 0, 0.5],
            [0, 0.5, 0.5]
        ])
    elif centering == 'C':
        return np.array([
            [0, 0, 0],
            [0.5, 0.5, 0]
        ])
    elif centering == 'A':
        return np.array([
            [0, 0, 0],
            [0, 0.5, 0.5]
        ])
    elif centering == 'B':
        return np.array([
            [0, 0, 0],
            [0.5, 0, 0.5]
        ])
    else:
        raise ValueError(f"Unknown centering type: {centering}")

# Display centering types
print("Bravais Lattice Centering Types:")
for centering in ['P', 'I', 'F', 'C']:
    points = get_centering_vectors(centering)
    print(f"\n{centering}: {len(points)} points per unit cell")
    for p in points:
        print(f"  {p}")

## 4.5 The Unit Cell Class

Let's create a comprehensive `UnitCell` class that handles all the mathematics of crystal lattices.

In [ ]:
class UnitCell:
    """A crystallographic unit cell.
    
    Handles conversion between fractional and Cartesian coordinates,
    lattice vector calculations, and volume computation.
    """
    
    def __init__(self, lattice_vectors: np.ndarray = None, 
                 params: LatticeParameters = None):
        """Initialize from either lattice vectors or parameters."""
        if lattice_vectors is not None:
            self._lattice_vectors = np.array(lattice_vectors, dtype=float)
        elif params is not None:
            self._lattice_vectors = lattice_vectors_from_parameters(params)
        else:
            # Default: unit cubic cell
            self._lattice_vectors = np.eye(3)
        
        # Cache the inverse for coordinate transformations
        self._inv_lattice_vectors = np.linalg.inv(self._lattice_vectors)
    
    @property
    def lattice_vectors(self) -> np.ndarray:
        """The 3x3 matrix of lattice vectors (rows are a, b, c)."""
        return self._lattice_vectors
    
    @property
    def a(self) -> np.ndarray:
        """First lattice vector."""
        return self._lattice_vectors[0]
    
    @property
    def b(self) -> np.ndarray:
        """Second lattice vector."""
        return self._lattice_vectors[1]
    
    @property
    def c(self) -> np.ndarray:
        """Third lattice vector."""
        return self._lattice_vectors[2]
    
    @property
    def parameters(self) -> LatticeParameters:
        """Lattice parameters."""
        return lattice_parameters_from_vectors(self._lattice_vectors)
    
    @property
    def volume(self) -> float:
        """Volume of the unit cell in Å³.
        
        V = a · (b × c) = det(lattice_vectors)
        """
        return abs(np.linalg.det(self._lattice_vectors))
    
    def fractional_to_cartesian(self, fractional: np.ndarray) -> np.ndarray:
        """Convert fractional coordinates to Cartesian.
        
        r_cartesian = f_a * a + f_b * b + f_c * c
                    = fractional @ lattice_vectors
        """
        fractional = np.atleast_2d(fractional)
        cartesian = fractional @ self._lattice_vectors
        return cartesian.squeeze() if cartesian.shape[0] == 1 else cartesian
    
    def cartesian_to_fractional(self, cartesian: np.ndarray) -> np.ndarray:
        """Convert Cartesian coordinates to fractional.
        
        fractional = cartesian @ inv(lattice_vectors)
        """
        cartesian = np.atleast_2d(cartesian)
        fractional = cartesian @ self._inv_lattice_vectors
        return fractional.squeeze() if fractional.shape[0] == 1 else fractional
    
    def wrap_to_unit_cell(self, fractional: np.ndarray) -> np.ndarray:
        """Wrap fractional coordinates to [0, 1) range."""
        return fractional % 1.0
    
    def minimum_image_distance(self, r1: np.ndarray, r2: np.ndarray) -> float:
        """Calculate minimum image distance under periodic boundary conditions.
        
        This accounts for the fact that atoms near cell boundaries
        may be closer through the periodic image.
        """
        # Convert to fractional
        f1 = self.cartesian_to_fractional(r1)
        f2 = self.cartesian_to_fractional(r2)
        
        # Find minimum image
        diff = f2 - f1
        diff = diff - np.round(diff)  # Wrap to [-0.5, 0.5)
        
        # Convert back to Cartesian and get distance
        diff_cart = self.fractional_to_cartesian(diff)
        return np.linalg.norm(diff_cart)
    
    def __repr__(self) -> str:
        params = self.parameters
        return (f"UnitCell(a={params.a:.3f}, b={params.b:.3f}, c={params.c:.3f}, "
                f"V={self.volume:.2f} Å³)")

# Create example unit cells
# FCC copper: a = 3.615 Å
cu_cell = UnitCell(params=LatticeParameters(3.615, 3.615, 3.615, 90, 90, 90))
print(f"FCC Copper: {cu_cell}")

# Graphite: a = 2.46 Å, c = 6.71 Å (hexagonal)
graphite_cell = UnitCell(params=LatticeParameters(2.46, 2.46, 6.71, 90, 90, 120))
print(f"Graphite: {graphite_cell}")

# Test coordinate conversion
print("\nCoordinate Conversion Test (Copper):")
frac_pos = np.array([0.5, 0.5, 0.5])  # Body center
cart_pos = cu_cell.fractional_to_cartesian(frac_pos)
frac_back = cu_cell.cartesian_to_fractional(cart_pos)

print(f"Fractional: {frac_pos}")
print(f"Cartesian:  {cart_pos}")
print(f"Back to fractional: {frac_back}")

## 4.6 The Metric Tensor

The **metric tensor** $G$ encodes all geometric information about the lattice:

$$G = \begin{pmatrix} \mathbf{a}\cdot\mathbf{a} & \mathbf{a}\cdot\mathbf{b} & \mathbf{a}\cdot\mathbf{c} \\
\mathbf{b}\cdot\mathbf{a} & \mathbf{b}\cdot\mathbf{b} & \mathbf{b}\cdot\mathbf{c} \\
\mathbf{c}\cdot\mathbf{a} & \mathbf{c}\cdot\mathbf{b} & \mathbf{c}\cdot\mathbf{c} \end{pmatrix}$$

Useful for:
- Computing distances from fractional coordinates: $d^2 = \Delta\mathbf{f}^T G \Delta\mathbf{f}$
- Computing angles
- Volume: $V = \sqrt{\det(G)}$

In [ ]:
def metric_tensor(cell: UnitCell) -> np.ndarray:
    """Compute the metric tensor G = M @ M^T where M is the lattice matrix."""
    M = cell.lattice_vectors
    return M @ M.T

def distance_from_fractional(f1: np.ndarray, f2: np.ndarray, G: np.ndarray) -> float:
    """Calculate distance between two points given in fractional coordinates.
    
    d² = Δf^T @ G @ Δf
    """
    df = f2 - f1
    d_squared = df @ G @ df
    return np.sqrt(d_squared)

# Example with hexagonal cell
G = metric_tensor(graphite_cell)
print("Metric tensor for graphite:")
print(np.round(G, 4))

# Verify: diagonal elements are a², b², c²
params = graphite_cell.parameters
print(f"\nExpected diagonal: [{params.a**2:.2f}, {params.b**2:.2f}, {params.c**2:.2f}]")
print(f"Actual diagonal:   {np.diag(G).round(2)}")

# Calculate distance using metric tensor
f1 = np.array([0, 0, 0])
f2 = np.array([1/3, 1/3, 0])  # A position in graphite structure

d_metric = distance_from_fractional(f1, f2, G)
d_cartesian = np.linalg.norm(
    graphite_cell.fractional_to_cartesian(f2) - 
    graphite_cell.fractional_to_cartesian(f1)
)

print(f"\nDistance using metric tensor: {d_metric:.4f} Å")
print(f"Distance using Cartesian:     {d_cartesian:.4f} Å")

## 4.7 The Reciprocal Lattice

The **reciprocal lattice** is fundamental to X-ray diffraction and electronic structure calculations.

### Definition

The reciprocal lattice vectors $\mathbf{a}^*, \mathbf{b}^*, \mathbf{c}^*$ satisfy:

$$\mathbf{a}_i \cdot \mathbf{a}_j^* = 2\pi \delta_{ij}$$

Explicit formulas:

$$\mathbf{a}^* = \frac{2\pi}{V} (\mathbf{b} \times \mathbf{c})$$
$$\mathbf{b}^* = \frac{2\pi}{V} (\mathbf{c} \times \mathbf{a})$$
$$\mathbf{c}^* = \frac{2\pi}{V} (\mathbf{a} \times \mathbf{b})$$

where $V = \mathbf{a} \cdot (\mathbf{b} \times \mathbf{c})$.

### Matrix Form

If $M$ is the matrix of direct lattice vectors (rows), then:

$$M^* = 2\pi (M^{-1})^T$$

In [ ]:
def reciprocal_lattice_vectors(cell: UnitCell) -> np.ndarray:
    """Calculate the reciprocal lattice vectors.
    
    Returns:
        3x3 matrix where rows are a*, b*, c* (in units of Å⁻¹ with 2π factor)
    """
    M = cell.lattice_vectors
    return 2 * np.pi * np.linalg.inv(M).T

def reciprocal_lattice_explicit(cell: UnitCell) -> np.ndarray:
    """Calculate reciprocal lattice using cross products (for verification)."""
    a, b, c = cell.a, cell.b, cell.c
    V = cell.volume
    
    a_star = 2 * np.pi * np.cross(b, c) / V
    b_star = 2 * np.pi * np.cross(c, a) / V
    c_star = 2 * np.pi * np.cross(a, b) / V
    
    return np.array([a_star, b_star, c_star])

# Calculate reciprocal lattice for copper
recip_cu = reciprocal_lattice_vectors(cu_cell)
recip_cu_explicit = reciprocal_lattice_explicit(cu_cell)

print("Reciprocal lattice vectors for FCC Copper (Å⁻¹):")
print("Matrix method:")
print(np.round(recip_cu, 4))
print("\nExplicit method:")
print(np.round(recip_cu_explicit, 4))

# Verify orthogonality: a · a* = 2π, a · b* = 0, etc.
print("\nVerification (should be 2π on diagonal, 0 elsewhere):")
orthog = cu_cell.lattice_vectors @ recip_cu.T
print(np.round(orthog, 6))
print(f"2π = {2*np.pi:.6f}")

## 4.8 Visualizing Lattices

In [ ]:
def plot_unit_cell(cell: UnitCell, ax: plt.Axes = None, 
                   centering: str = 'P', title: str = None):
    """Plot a unit cell in 3D."""
    if ax is None:
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(111, projection='3d')
    
    # Get lattice vectors
    a, b, c = cell.a, cell.b, cell.c
    
    # Draw the unit cell edges
    origin = np.array([0, 0, 0])
    vertices = np.array([
        origin,
        a,
        a + b,
        b,
        c,
        a + c,
        a + b + c,
        b + c
    ])
    
    # Edges to draw
    edges = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # Bottom face
        (4, 5), (5, 6), (6, 7), (7, 4),  # Top face
        (0, 4), (1, 5), (2, 6), (3, 7)   # Vertical edges
    ]
    
    for i, j in edges:
        ax.plot3D(*zip(vertices[i], vertices[j]), 'b-', linewidth=1)
    
    # Plot lattice points based on centering
    centering_fracs = get_centering_vectors(centering)
    for frac in centering_fracs:
        cart = cell.fractional_to_cartesian(frac)
        ax.scatter(*cart, s=100, c='red', edgecolors='black')
    
    # Also plot corner points (translated by lattice vectors)
    for i in range(2):
        for j in range(2):
            for k in range(2):
                corner = cell.fractional_to_cartesian(np.array([i, j, k]))
                ax.scatter(*corner, s=80, c='blue', alpha=0.5)
    
    # Draw lattice vector arrows
    colors = ['red', 'green', 'blue']
    labels = ['a', 'b', 'c']
    for vec, color, label in zip([a, b, c], colors, labels):
        ax.quiver(0, 0, 0, *vec, color=color, arrow_length_ratio=0.1, linewidth=2)
        ax.text(*(vec/2), label, fontsize=12, color=color)
    
    ax.set_xlabel('X (Å)')
    ax.set_ylabel('Y (Å)')
    ax.set_zlabel('Z (Å)')
    ax.set_title(title or f'Unit Cell ({centering} centering)')
    
    return ax

# Plot different centerings for cubic cell
fig = plt.figure(figsize=(15, 5))

cubic = UnitCell(params=LatticeParameters(4, 4, 4, 90, 90, 90))

for i, (centering, name) in enumerate([('P', 'Primitive'), ('I', 'Body-centered'), ('F', 'Face-centered')]):
    ax = fig.add_subplot(1, 3, i+1, projection='3d')
    plot_unit_cell(cubic, ax, centering=centering, title=f'{name} Cubic')

plt.tight_layout()
plt.show()

## 4.9 Generating Lattice Points

To build a supercell or visualize a lattice, we need to generate lattice points.

In [ ]:
def generate_lattice_points(cell: UnitCell, 
                            n_cells: Tuple[int, int, int] = (3, 3, 3),
                            centering: str = 'P',
                            centered: bool = True) -> np.ndarray:
    """Generate lattice points for a supercell.
    
    Args:
        cell: Unit cell
        n_cells: Number of cells in each direction (na, nb, nc)
        centering: Lattice centering type
        centered: If True, center the points around origin
    
    Returns:
        Nx3 array of Cartesian positions
    """
    na, nb, nc = n_cells
    centering_vecs = get_centering_vectors(centering)
    
    points = []
    
    # Generate grid of cells
    for i in range(na):
        for j in range(nb):
            for k in range(nc):
                # Translation vector for this cell
                cell_origin = np.array([i, j, k], dtype=float)
                
                # Add all centering points
                for cv in centering_vecs:
                    frac = cell_origin + cv
                    cart = cell.fractional_to_cartesian(frac)
                    points.append(cart)
    
    points = np.array(points)
    
    if centered:
        # Center around geometric mean
        center = cell.fractional_to_cartesian(
            np.array([(na-1)/2, (nb-1)/2, (nc-1)/2])
        )
        points -= center
    
    return points

# Generate FCC lattice points
fcc_points = generate_lattice_points(cu_cell, (3, 3, 3), centering='F')

print(f"Generated {len(fcc_points)} FCC lattice points")

# Visualize
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(fcc_points[:, 0], fcc_points[:, 1], fcc_points[:, 2], 
           s=50, c='copper', edgecolors='black', alpha=0.8)

ax.set_xlabel('X (Å)')
ax.set_ylabel('Y (Å)')
ax.set_zlabel('Z (Å)')
ax.set_title('FCC Copper Lattice (3×3×3 supercell)')

plt.show()

---

## Practice Exercises

### Exercise 4.1: Primitive Cell of BCC
The conventional unit cell of BCC is not primitive (it has 2 atoms). Find the primitive lattice vectors.

In [ ]:
# YOUR CODE HERE
def bcc_primitive_cell(a: float) -> UnitCell:
    """Create the primitive unit cell for a BCC lattice.
    
    The primitive vectors connect a corner to the three nearest
    body centers in adjacent cells.
    
    a1 = a/2 * (1, 1, -1)
    a2 = a/2 * (1, -1, 1)
    a3 = a/2 * (-1, 1, 1)
    """
    # TODO: Implement
    pass

### Exercise 4.2: Nearest Neighbor Distance
Calculate the nearest-neighbor distance for FCC, BCC, and simple cubic lattices.

In [ ]:
# YOUR CODE HERE
def nearest_neighbor_distance(cell: UnitCell, centering: str = 'P') -> float:
    """Calculate the nearest-neighbor distance for a lattice.
    
    Hint: Generate centering points and find minimum distance.
    """
    # TODO: Implement
    pass

### Exercise 4.3: Supercell Matrix
Implement a general supercell transformation using an integer matrix.

In [ ]:
# YOUR CODE HERE
def make_supercell(cell: UnitCell, P: np.ndarray) -> UnitCell:
    """Create a supercell using transformation matrix P.
    
    The new lattice vectors are: A' = P @ A
    where A is the 3x3 matrix of original lattice vectors.
    
    Volume scales as: V' = |det(P)| * V
    
    Args:
        cell: Original unit cell
        P: 3x3 integer transformation matrix
    """
    # TODO: Implement
    pass

### Exercise 4.4: Wigner-Seitz Cell
The Wigner-Seitz cell is the primitive cell with maximum symmetry. Describe how to construct it.

In [ ]:
# YOUR CODE HERE
def is_inside_wigner_seitz(point: np.ndarray, cell: UnitCell) -> bool:
    """Check if a point is inside the Wigner-Seitz cell.
    
    The WS cell consists of all points closer to the origin
    than to any other lattice point.
    
    Hint: Check distance to origin vs distance to nearby lattice points.
    """
    # TODO: Implement
    pass

### Exercise 4.5: Reciprocal Lattice of Hexagonal
Show that the reciprocal lattice of a hexagonal lattice is also hexagonal (rotated by 30°).

In [ ]:
# YOUR CODE HERE
# Create a hexagonal cell
hex_cell = UnitCell(params=LatticeParameters(3.0, 3.0, 5.0, 90, 90, 120))

# TODO: Calculate reciprocal lattice and verify it's hexagonal
# TODO: Find the rotation angle between direct and reciprocal lattice

---

## Key Takeaways

1. **Lattice vectors** $\mathbf{a}, \mathbf{b}, \mathbf{c}$ define the periodic structure
2. **7 crystal systems** arise from symmetry constraints on lattice parameters
3. **14 Bravais lattices** include different centering types (P, I, F, C)
4. **Fractional coordinates** are natural for periodic systems
5. **The metric tensor** enables distance calculations from fractional coordinates
6. **The reciprocal lattice** is essential for diffraction and electronic structure

## Next Chapter Preview

In Chapter 5, we'll explore **Building Crystal Structures**:
- Combining lattice and basis
- Common crystal structures (FCC, BCC, diamond, rocksalt, etc.)
- Space groups and Wyckoff positions
- Reading crystallographic data